In [2]:
import numpy as np
from glob import glob
import os
from os import listdir

In [3]:
def read_roi(fileobj):
    '''
    points = read_roi(fileobj)
    Read ImageJ's ROI format
    '''
# This is based on:
# http://rsbweb.nih.gov/ij/developer/source/ij/io/RoiDecoder.java.html
# http://rsbweb.nih.gov/ij/developer/source/ij/io/RoiEncoder.java.html


    SPLINE_FIT = 1
    DOUBLE_HEADED = 2
    OUTLINE = 4
    OVERLAY_LABELS = 8
    OVERLAY_NAMES = 16
    OVERLAY_BACKGROUNDS = 32
    OVERLAY_BOLD = 64
    SUB_PIXEL_RESOLUTION = 128
    DRAW_OFFSET = 256


    pos = [4]
    def get8():
        pos[0] += 1
        s = fileobj.read(1)
        #if not s:
        #    raise IOError('readroi: Unexpected EOF')
        return ord(s)

    def get16():
        b0 = get8()
        b1 = get8()
        return (b0 << 8) | b1

    def get32():
        s0 = get16()
        s1 = get16()
        return (s0 << 16) | s1

    def getfloat():
        v = np.int32(get32())
        return v.view(np.float32)

    magic = fileobj.read(4)
    if magic != 'Iout':
        raise IOError('Magic number not found')
    version = get16()

    # It seems that the roi type field occupies 2 Bytes, but only one is used
    roi_type = get8()
    # Discard second Byte:
    get8()

    if not (0 <= roi_type < 11):
        raise ValueError('roireader: ROI type %s not supported' % roi_type)

    #if roi_type != 7:
        #raise ValueError('roireader: ROI type %s not supported (!= 7)' % roi_type)

    top = get16()
    left = get16()
    bottom = get16()
    right = get16()
    n_coordinates = get16()

    x1 = getfloat() 
    y1 = getfloat() 
    x2 = getfloat() 
    y2 = getfloat()
    stroke_width = get16()
    shape_roi_size = get32()
    stroke_color = get32()
    fill_color = get32()
    subtype = get16()
    if subtype != 0:
        raise ValueError('roireader: ROI subtype %s not supported (!= 0)' % subtype)
    options = get16()
    arrow_style = get8()
    arrow_head_size = get8()
    rect_arc_size = get16()
    position = get32()
    header2offset = get32()


    if options & SUB_PIXEL_RESOLUTION:
        getc = getfloat
        points = np.empty((n_coordinates, 2), dtype=np.float32)
    else:
        getc = get16
        points = np.empty((n_coordinates, 2), dtype=np.int16)

    #points[:,0] = [getc() for i in range(n_coordinates)]
    #points[:,1] = [getc() for i in range(n_coordinates)]
    #points[:,0] += left
    #points[:,1] += top
    #points -= 1
    return [top,bottom,left,right]

In [4]:
folders_composite=glob("Composite Images/*/")
folders_composite

['Composite Images\\20190508-C2GNT1-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-05-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-05-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-GALNT

In [5]:
folders_composite[0][17:len(folders_composite[0])-18]

'20190508-C2GNT1-EGFP-Noc-01'

In [6]:
folder_current=glob("*/")
folder_current.remove('Composite Images\\')
folder_current

['20190508-C2GNT1-EGFP-Noc-01\\',
 '20190508-C2GNT1-EGFP-Noc-02\\',
 '20190508-C2GNT1-EGFP-Noc-03\\',
 '20190508-C2GNT1-EGFP-Noc-04\\',
 '20190508-C2GNT1-EGFP-Noc-05\\',
 '20190508-GALNT4-EGFP-Noc-01\\',
 '20190508-GALNT4-EGFP-Noc-02\\',
 '20190508-GALNT4-EGFP-Noc-03\\',
 '20190508-GALNT4-EGFP-Noc-04\\',
 '20190508-GALNT4-EGFP-Noc-05\\',
 '20190508-GALNT8-EGFP-Noc-01\\',
 '20190508-GALNT8-EGFP-Noc-02\\',
 '20190508-GALNT8-EGFP-Noc-03\\',
 '20190508-GALNT8-EGFP-Noc-04\\',
 '20190508-GALNT8-EGFP-Noc-05\\',
 '20190508-GALNT8-EGFP-Noc-06\\',
 '20190508-GALNT8-EGFP-Noc-07\\',
 '20190508-GALNT8-EGFP-Noc-08\\',
 '20190508-tetraspeckbeads-aft-01\\',
 '20190508-tetraspeckbeads-bef-01\\',
 '20190508-VIP36-EGFP-Noc-01\\',
 '20190508-VIP36-EGFP-Noc-02\\',
 '20190508-VIP36-EGFP-Noc-03\\',
 '20190508-VIP36-EGFP-Noc-04\\',
 '20190508-VIP36-EGFP-Noc-05\\',
 '20190508-VIP36-EGFP-Noc-06\\',
 '20190508-VIP36-EGFP-Noc-07\\',
 '20190508-VIP36-EGFP-Noc-08\\',
 '20190823-EGFP-Golph3-Noc-01\\',
 '20190823-EGF

In [7]:
folder_current[0][0:len(folder_current[0])-1]

'20190508-C2GNT1-EGFP-Noc-01'

In [8]:
os.getcwd()

'e:\\MBDS Materials\\BMDSIS\\Data'

In [9]:
listdir([i for i in glob(folder_current[0][0:len(folder_current[2])-1]+'/*/') if 'RoiSet' in i][0])

['0015-0116.roi',
 '0104-0120.roi',
 '0162-0336.roi',
 '0225-0242.roi',
 '0236-0112.roi',
 '0259-0053.roi',
 '0291-0231.roi',
 '0300-0095.roi',
 '0303-0049.roi',
 '0306-0298.roi',
 '0306-1093.roi',
 '0322-1129.roi',
 '0331-1076.roi',
 '0338-0047.roi',
 '0341-0394.roi',
 '0342-0575.roi',
 '0365-1177.roi',
 '0366-0025.roi',
 '0369-1130.roi',
 '0389-0303.roi',
 '0396-1175.roi',
 '0502-1146.roi',
 '0523-0560.roi',
 '0532-0377.roi',
 '0556-1169.roi',
 '0587-0369.roi',
 '0629-0335.roi',
 '0636-0081.roi',
 '0661-1146.roi',
 '0683-1048.roi',
 '0718-1045.roi',
 '0735-0077.roi',
 '0770-0116.roi',
 '0799-0923.roi',
 '0801-0976.roi',
 '0847-0166.roi']

In [10]:
import pandas as pd

In [11]:
read_roi(open("E:\\MBDS Materials\\BMDSIS\\Data\\20190508-C2GNT1-EGFP-Noc-01\\20190508-C2GNT1-EGFP-01-RoiSet\\0015-0116.roi",encoding='latin-1'))

[2, 28, 102, 131]

In [12]:
roi_stats=pd.DataFrame(columns=['label','roi_file','x_length','y_length'])

In [13]:
for f in folder_current:
    label=f[0:len(f)-1]
    roi_path=os.getcwd()+"\\"+[i for i in glob(f[0:len(f)-1]+'/*/') if 'RoiSet' in i][0]
    roi_files=listdir(roi_path)
    
    point_df=pd.DataFrame(columns=['top','bottom','left','right'])

    for rf in roi_files:
        print(rf)

        points_info=read_roi(open(roi_path+"\\"+rf,encoding='latin-1'))
        insert_row=pd.DataFrame([points_info],columns=['top','bottom','left','right'])
        point_df=pd.concat([point_df,insert_row])

        insert_row_roi_stats=pd.DataFrame([[label,rf,points_info[3]-points_info[2],points_info[1]-points_info[0]]],columns=['label','roi_file','x_length','y_length'])
        roi_stats=pd.concat([roi_stats,insert_row_roi_stats]).reset_index(drop=True)
    
    point_df=point_df.reset_index(drop=True)
    point_df.to_csv(os.getcwd()+"\\Composite Images\\"+label+"-Data Preparation\\"+label+"-ROI Coordinates.csv")

108.roi
0317-0894.roi
0317-0914.roi
0339-0921.roi
0354-0955.roi
0368-0922.roi
0386-0534.roi
0388-0616.roi
0393-0506.roi
0400-1069.roi
0437-0882.roi
0442-1030.roi
0464-0897.roi
0482-0952.roi
0529-0861.roi
0546-0958.roi
0697-0854.roi
0730-0895.roi
0255-0603.roi
0277-0405.roi
0338-0623.roi
0362-0244.roi
0371-0220.roi
0391-0199.roi
0412-0160.roi
0457-0446.roi
0472-0210.roi
0474-0538.roi
0505-0537.roi
0514-0600.roi
0518-0558.roi
0539-0581.roi
0548-0469.roi
0553-0555.roi
0570-0492.roi
0596-0324.roi
0596-0360.roi
0638-0520.roi
0959-0478.roi
0988-0602.roi
0177-0298.roi
0201-0406.roi
0242-0263.roi
0310-0265.roi
0601-0939.roi
0664-0949.roi
0681-0974.roi
0759-0730.roi
0771-0759.roi
0786-0804.roi
0860-0916.roi
0908-0854.roi
0068-1126.roi
0095-1126.roi
0162-1078.roi
0265-0974.roi
0269-1009.roi
0382-1067.roi
0411-0873.roi
0425-1138.roi
0427-1066.roi
0458-1072.roi
0477-0951.roi
0504-0869.roi
0634-0454.roi
0660-0416.roi
0669-0535.roi
0685-0461.roi
0707-1095.roi
0739-0437.roi
0760-0365.roi
0762-0455.ro

In [14]:
os.getcwd()+"\\Composite Images\\"+label+"-Data Preparation\\"

'e:\\MBDS Materials\\BMDSIS\\Data\\Composite Images\\20190823-tetraspeckbeads-bef-01-Data Preparation\\'

In [15]:
roi_stats

,label,roi_file,x_length,y_length
0,20190508-C2GNT1-EGFP-Noc-01,0015-0116.roi,29,26
1,20190508-C2GNT1-EGFP-Noc-01,0104-0120.roi,17,21
2,20190508-C2GNT1-EGFP-Noc-01,0162-0336.roi,28,30
3,20190508-C2GNT1-EGFP-Noc-01,0225-0242.roi,29,32
4,20190508-C2GNT1-EGFP-Noc-01,0236-0112.roi,44,33
...,...,...,...,...
3752,20190823-tetraspeckbeads-bef-01,1058-0412.roi,15,15
3753,20190823-tetraspeckbeads-bef-01,1060-1207.roi,15,15
3754,20190823-tetraspeckbeads-bef-01,1063-0723.roi,15,15
3755,20190823-tetraspeckbeads-bef-01,1066-1263.roi,15,15


In [16]:
roi_stats['y_length'].count()

3757

In [17]:
roi_stats['y_length']

0       26
1       21
2       30
3       32
4       33
        ..
3752    15
3753    15
3754    15
3755    15
3756    15
Name: y_length, Length: 3757, dtype: object

In [18]:
roi_stats['y_length'].value_counts()

15    2634
18     123
19     123
20     122
17     108
21      88
22      72
16      69
23      65
25      54
24      52
14      46
26      36
13      29
27      26
12      25
29      15
28      14
30      10
31       8
11       7
32       7
33       7
34       7
36       3
35       2
37       2
40       1
10       1
41       1
Name: y_length, dtype: int64

In [19]:
roi_stats['x_length'].value_counts()

15    2628
21     110
18     103
19     102
20      97
16      84
23      77
17      73
22      70
24      67
25      62
26      46
14      41
27      31
28      29
12      23
29      18
30      18
13      16
33      14
31      12
32      10
34       7
38       3
11       3
35       3
36       3
10       2
37       2
47       2
44       1
Name: x_length, dtype: int64

In [20]:
roi_stats['x_length']=roi_stats['x_length'].apply(float)
roi_stats['y_length']=roi_stats['y_length'].apply(float)

In [21]:
roi_stats['x_length'].describe()

count    3757.000000
mean       16.892467
std         3.988821
min        10.000000
25%        15.000000
50%        15.000000
75%        17.000000
max        47.000000
Name: x_length, dtype: float64

In [22]:
roi_stats['y_length'].describe()

count    3757.000000
mean       16.612457
std         3.528514
min        10.000000
25%        15.000000
50%        15.000000
75%        17.000000
max        41.000000
Name: y_length, dtype: float64

In [23]:
roi_stats['x_length'].mean()

16.8924673941975

In [24]:
roi_stats['x_length'].median()

15.0

In [25]:
roi_stats['y_length'].mean()

16.612456747404845

In [26]:
roi_stats['y_length'].median()

15.0